## Exploring different Document Processing Models
 
 **Goal**: An Agentic approach to explore different Document Processing Models for OCR, Layout Analysis, and Table Extraction.
 
 **Output**: A simple langgraph agent that explores different ocr techniques/models.


In [ ]:
# Setting up a react agent with LangChain


from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.messages import AIMessageChunk, HumanMessage
from langchain_core.runnables import RunnableConfig

# from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.checkpoint.memory import InMemorySaver
from langsmith import traceable
from prompts import DOCUMENT_AGENT_SYSTEM_PROMPT, USER_MESSAGE_TEMPLATE

load_dotenv()
# conn = connect("langgraph_checkpoints.db")
# checkpointer = SqliteSaver(conn=conn)
checkpointer = InMemorySaver()
llm = init_chat_model(
    "anthropic:claude-sonnet-4-5-20250929", temperature=1, thinking={"type": "enabled", "budget_tokens": 10000}
)

document_agent = create_agent(
    model=llm,
    system_prompt=DOCUMENT_AGENT_SYSTEM_PROMPT,
    checkpointer=checkpointer,
)
config = RunnableConfig(configurable={"thread_id": "test_thread"})
inputs = {
    "messages": [
        HumanMessage(
            USER_MESSAGE_TEMPLATE.format(message="What OCR tools are available?", document_path="No Document Uploaded")
        )
    ]
}


@traceable(name="DocumentAgentInteraction")
def agent_execution():
    """Execute the document agent and streams the response."""
    ai_message = AIMessageChunk(content=[])

    for _, chunk in document_agent.stream(inputs, stream_mode=["messages"], config=config):
        ai_message += chunk[0]

    ai_message.pretty_print()
    return ai_message


ai_message = agent_execution()

============================ Aimessagechunk Message ============================

[{'thinking': 'The user is asking about what OCR tools are available. Looking at the prompt, there\'s a placeholder for `{available_ocr_tools}` but it hasn\'t been filled in with actual tools. \n\nThe user also mentions they uploaded a document, but the system message says "No Document Uploaded".\n\nLet me address both points:\n1. List the available OCR tools (though the placeholder suggests they should be dynamically provided)\n2. Note that no document has been uploaded yet\n\nSince the `{available_ocr_tools}` placeholder is empty, I should inform the user that the OCR tools information should be configured in the system, but I can mention common OCR techniques that are typically available in document processing systems.', 'type': 'thinking', 'index': 0, 'signature': 'EucGCkYICxgCKkC6hJnbW1Rl9K7jIJ/voIvbEv2U0R54MH9ts4NZgGrpTOcGkKk1mtZeYSYfgIgmxihEtaf+hCkM4O+u9aHsqnJvEgx3x6Ra9BbntIETfLgaDNLiXcSA/XSPf57mxC

'lc_run--019bc357-e58a-75f1-a744-e292e0334d75'

In [48]:
from datetime import datetime
from uuid import UUID

uuid = UUID(ai_message.id.split("--")[1])
ts_ms = uuid.int >> 80
ts_s = ts_ms / 1000
dt = datetime.fromtimestamp(ts_s)
print("Message created at:", dt)

Message created at: 2026-01-16 01:27:56.170000


In [ ]:
from langchain.messages import AIMessage

AIMessage()